In [29]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split

# Read the CSV and Perform Basic Data Cleaning

In [30]:
df_koi = pd.read_csv("cumulative.csv")
df_koi = df_koi.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])

# Drop the null columns where all values are null
df_koi = df_koi.dropna(axis='columns', how='all')

# Drop the null rows
df_koi = df_koi.dropna()
df_koi.head()


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [31]:
# Transforming non-numerical to numerical data 
le = preprocessing.LabelEncoder() 
koi_disposition = le.fit_transform(list(df_koi['koi_disposition']))

As data was assigned in to a list while transforming non-numerical to
numerical data, list must be turn into a numpy array to proceed with 
the data transfornation process.   




In [32]:
# Assigning X (data) and y (target) while turning y into a numpy array in order 
# to reshape

X = df_koi[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co','koi_fpflag_ec', 
            'koi_period', 'koi_period_err1', 'koi_period_err2','koi_time0bk', 
            'koi_time0bk_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 
            'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 
            'ra', 'dec','koi_kepmag'
           ]]
y = np.array(koi_disposition).reshape(-1, 1)

print(x.shape, y.shape)

(8744, 19) (8744, 1)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [33]:
# Spliting data into training and testing. 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [34]:
x_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
1054,0,1,1,1,0.520223,0.000001,-0.000001,178.148460,0.001800,-173.0,4.527,0.038,-0.212,0.889,0.273,-0.091,297.04651,49.970390,15.632
86,0,0,0,0,46.184204,0.000265,-0.000265,165.237290,0.004300,-86.0,4.754,0.042,-0.031,0.493,0.033,-0.040,286.07913,39.278320,15.954
8029,0,0,0,1,1.752516,0.000033,-0.000033,132.754400,0.024200,-160.0,4.554,0.033,-0.176,0.844,0.226,-0.071,299.16608,41.863049,13.978
1794,0,0,0,0,91.500873,0.000523,-0.000523,165.673090,0.004930,-90.0,4.628,0.017,-0.052,0.706,0.047,-0.030,296.85101,50.698929,14.447
2623,0,0,0,0,13.748831,0.000013,-0.000013,179.279158,0.000738,-80.0,4.050,0.033,-0.027,1.670,0.118,-0.082,289.08606,41.562958,11.565


# Pre-processing

Scale the data using the MinMaxScaler

In [35]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler().fit(x_train)
y_scaler = StandardScaler().fit(y_train)


# Train the Support Vector Machine

In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the `C` and `gamma` parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)